In [1]:
import cvxopt as opt
from cvxopt import matrix,solvers
import numpy as np
import math

In [2]:
from dataset import *

In [3]:
class svm_kernel:
    kernelfuc=None
    b=0
    alpha,SV,lam,y=[],[],[],[]
    def kernel_null(self,x1,x2):#线性
        return x1.T.dot(x2)
    def kernel_theta2(self,x1,x2):#多项式
        line=x1.T.dot(x2)
        return math.pow(self.lam[0]+self.lam[1]*line,self.lam[2])
    def kernel_goss(self,x1,x2):#高斯
        line=-1*self.lam[0]*(x1-x2).T.dot(x1-x2)
        return math.exp(line)
    def svm_train(self,train_data,train_label,kernel=("line",0.00001)):
        def creatqt(train_data,train_label,kernelfuc):#创建QP条件
            P=np.zeros([train_data.shape[0],train_data.shape[0]])
            for c in range(train_data.shape[0]):
                for b in range(train_data.shape[0]):
                    P[c][b]=train_label[c][0]*train_label[b][0]*kernelfuc(train_data[c],train_data[b])
            q=np.ones(shape=[train_data.shape[0],1])*-1
            A=train_label.reshape([1,train_data.shape[0]])*1.0
            h=np.zeros([train_data.shape[0],1])
            b=0.0
            G=np.eye(train_data.shape[0])*-1
            return matrix(q),matrix(P),matrix(A,),matrix(b),matrix(G),matrix(h)
        def get_alphe_w_b(self,opt,train_data,train_label,marger):
            index=list(np.where(opt>marger)[0])
            for b in index:
                self.alpha.append(opt[b])
                self.SV.append(train_data[b])
                self.y.append(train_label[b])
            indexmax=opt.argmax()
            legth=train_data.shape[0]*train_data.shape[1]
            xs,ys=train_data[indexmax],train_label[indexmax]
            for b in range(len(index)):
                self.b=self.b+self.y[b]*self.alpha[b]*self.kernelfuc(self.SV[b],xs)
            self.b=ys-self.b
        self.kernelfuc=None
        self.b=0
        self.alpha,self.SV,self.lam,self.y=[],[],[],[]
        if kernel[0]=="line":
            self.kernelfuc=self.kernel_null
        if kernel[0]=="thetaQ":
            self.kernelfuc=self.kernel_theta2
            self.lam.append(kernel[1])
            self.lam.append(kernel[2])
            self.lam.append(kernel[3])
        if kernel[0]=="goss":
            self.kernelfuc=self.kernel_goss
            self.lam.append(kernel[1])
        q,P,A,b,G,h=creatqt(train_data,train_label,self.kernelfuc)
        opt=np.array(solvers.qp(G=G,P=P,q=q,h=h,A=A,b=b)['x'])
        get_alphe_w_b(self,opt,train_data,train_label,kernel[-1])
        print(opt)
    def pre(self,test_data):
        sig=0
        for b,c,k in zip(self.SV,self.alpha,self.y):
            sig=sig+self.kernelfuc(b,test_data)*c*k
        return np.sign(sig+self.b)

In [5]:
data,label=data_build(size=[10,10,2],label=("list",[1,-1]),sdv=[[-5,0],[5,0]])
svm=svm_kernel()
a=svm.svm_train(train_data=data,train_label=label,kernel=(["line",0.1]))

     pcost       dcost       gap    pres   dres
 0: -7.6108e-01 -1.0817e+00  3e+01  6e+00  1e+00
 1: -1.3378e-01 -1.2613e-01  2e+00  3e-01  6e-02
 2: -5.4151e-03 -1.0129e-01  1e-01  5e-17  7e-16
 3: -3.1418e-02 -4.2161e-02  1e-02  8e-18  3e-16
 4: -3.8858e-02 -4.0691e-02  2e-03  5e-18  3e-16
 5: -4.0003e-02 -4.0043e-02  4e-05  3e-17  3e-16
 6: -4.0029e-02 -4.0030e-02  4e-07  8e-18  3e-16
 7: -4.0030e-02 -4.0030e-02  4e-09  2e-17  2e-16
Optimal solution found.
[[4.42422630e-10]
 [4.40593355e-10]
 [6.29222587e-09]
 [2.33995557e-10]
 [1.33203025e-02]
 [2.67094023e-02]
 [2.36859333e-10]
 [5.33604829e-10]
 [2.27519352e-10]
 [5.99952264e-10]
 [6.39976788e-10]
 [2.74723756e-10]
 [2.37330675e-10]
 [2.57870282e-10]
 [4.00297106e-02]
 [2.38320544e-10]
 [6.11830533e-10]
 [2.34202310e-10]
 [2.40264919e-10]
 [5.26544796e-10]]


In [ ]:
svm.pre(test_data=np.array([12,25]))

In [7]:
chinamap={'台北':[121,23],'台州':[121,28],'温州':[120,28],'北京':[116,39]}
japanmap={'鹿儿岛':[131,30],'大阪':[135,34],'广岛':[132,34],'奈良':[135,34]}

In [8]:
train_data,train_label=[],[]
for b in chinamap:
    train_data.append(chinamap[b])
    train_label.append([-1])
for b in japanmap:
    train_data.append(japanmap[b])
    train_label.append([1])
train_data=np.array(train_data)
train_label=np.array(train_label)

In [14]:
svm=svm_kernel()
svm.svm_train(train_data=train_data,train_label=train_label,kernel=['goss',3,0.0001])
pre()
svm.svm_train(train_data=train_data,train_label=train_label,kernel=['line',0.0001])
pre()
svm.svm_train(train_data=train_data,train_label=train_label,kernel=['thetaQ',1,1,5,0])
pre()

     pcost       dcost       gap    pres   dres
 0: -2.6418e+00 -6.5520e+00  4e+00  2e-16  2e+00
 1: -3.2140e+00 -3.4082e+00  2e-01  5e-16  3e-01
 2: -3.3932e+00 -3.4051e+00  1e-02  2e-16  9e-17
 3: -3.3932e+00 -3.3934e+00  1e-04  6e-16  8e-17
 4: -3.3932e+00 -3.3933e+00  1e-06  4e-16  7e-17
Optimal solution found.
[[0.86891705]
 [0.8277079 ]
 [0.8277079 ]
 [0.86891705]
 [1.13108329]
 [0.56554166]
 [1.13108329]
 [0.56554166]]
钓鱼带是中国的
     pcost       dcost       gap    pres   dres
 0: -3.7349e-01 -5.6274e-01  1e+01  3e+00  1e+00
 1: -1.0762e-01 -7.7882e-02  1e+00  3e-01  1e-01
 2:  7.6674e-04 -5.6894e-02  6e-02  5e-17  4e-13
 3: -1.5301e-02 -2.1085e-02  6e-03  4e-18  6e-14
 4: -1.8896e-02 -1.9463e-02  6e-04  4e-18  6e-14
 5: -1.9225e-02 -1.9233e-02  8e-06  3e-18  5e-14
 6: -1.9231e-02 -1.9231e-02  8e-08  3e-18  3e-14
Optimal solution found.
[[1.13965687e-07]
 [1.92306035e-02]
 [3.49231438e-08]
 [1.36572349e-09]
 [1.92307112e-02]
 [1.17384498e-08]
 [1.91476613e-08]
 [1.17384498e-08]]
钓鱼

In [10]:
def pre():
    predict=svm.pre(test_data=np.array([124,26]))
    if(predict==-1):
        print('钓鱼带是中国的')
    else:
        print('钓鱼岛是日本的')